<img src="logo2.png"></img>

# The Final Challenge: Sample Solution & Honorary Mentions

# What was the final challenge about?
Zed City is a newly established (fictitious) municipality in Tokyo and is made up of 11 districts. <br/> 
Four convenience store ( _konbini_ ) chains A, B, C and D have each established their first store in this new city in non-overlapping districts.
The map (or graph) below shows the 11 districts of Zed City and which district has a konbini already.
<img src="./tokyo_map_pic.png" width="700">
As the mayor of Zed City, you want to establish konbinis in the rest of the districts that still don't have one yet.<br/>
Upon your request, all four konbini chains discussed with each other and agreed to establish their konbinis in Zed City under the following two conditions:

**-Only one konbini is allowed in one district.**<br/>
**-No two adjacent districts can have konbinis from the same chain.**<br/>

Can you come up with a plan that satisfies the above conditions? You must provide all store plan combinations that meet the conditions above?
(For details of the problem, visit [final challenge](https://github.com/quantum-challenge/2019/tree/master/problems/final).)

# A Sample Solution
(This is only a sample solution. There are many clever ways to solve this problem as you will see in the top teams' submissions. So just remember to look at this solution literally as *a sample* and not as the only correct answer.) <br/><br/>
The Final Challenge is larger in scale compared to any of the learning challenges, however, it can be solved with the same algorithm (Grover) we learned in week 2, which can be summarized in the following four steps.

|<p align="left">**Step**  |<p align="left">**Content** |
| ---     |--- |
| Step1 |<p align="left">Create a superposition of inputs|
| Step2 |<p align="left">Build an oracle according to the constraints of the problem |
| Step3  |<p align="left">Diffusion |
| Step4  |<p align="left">Measurement|

## Quantum Registers
Let's take a look at an example of how we can assign our inputs into our quantum registers. 

|Qubit|Number of qubits|Assignment|
|:---:|:---:|:---:|
|q[0]-q[13]|14|To store konbini info. for all 7 empty districts ($\log_2{4}$ per district)|
|q[14]-q[26]|13|Data storage inside oracle|
|q[27]|1|To store the oracle result|
|q[28]-q[31]|4|Ancilla qubits for oracle|


## Step1: Create a superposition of inputs

In [1]:
#prepare our quantum circuit by importing relevant modules
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, execute

qr = QuantumRegister(32)
cr = ClassicalRegister(14)
qc = QuantumCircuit(qr,cr)

We first create a superposition of konbinis across q[0]~q[13] for all possible combinations where A:00, B:01, C:10, D11. 

The first key optimization is to reduce the size of your solution space by narrowing down superposition of states in districts that already have kombinis A, B, C or D adjacent to them.

For example, district 3 is neighbored by konbini A, so you can eliminate the $|00>$ state and create a superposition of $|01>$, $|10>$ and $|11>$ only to initialize this district.

Similarly, district 2 is neighbored by konbini A and C thus, you can eliminate 
$|00>$ and $|10>$ and create a superposition of $|01>$ and $|11>$ only to initialize this district.

Based on the same idea, you should be able to initialize each district as follows:

In [2]:
theta = 2 * np.arccos(1 / np.sqrt(3))

In [3]:
def initialize_districts(qc, qr):
    #district0 A 
    qc.ry(theta,qr[0])
    qc.ch(qr[0],qr[1])
    qc.x(qr[1])

    #district1 B
    qc.ry(theta,qr[2])
    qc.ch(qr[2],qr[3])

    #district2 A,C
    qc.h(qr[4])
    qc.x(qr[5])

    #district3 A
    qc.ry(theta,qr[6])
    qc.ch(qr[6],qr[7])
    qc.x(qr[7])

    #district4 B
    qc.ry(theta,qr[8])
    qc.ch(qr[8],qr[9])

    #district5 D
    qc.ry(theta,qr[10])
    qc.ch(qr[10],qr[11])
    qc.x(qr[10])

    #district6 D
    qc.ry(theta,qr[12])
    qc.ch(qr[12],qr[13])
    qc.x(qr[12])

## Step2: Build an oracle according to the constraints of the problem

As we have taken care of the input states for districts that have neighboring konbinis in Step 1, we can focus on the remaining districts which gives us a graph representation as follows with 13 edges. 

<img src="./fig/white_map.png" width="400"></img>

In [4]:
edge_list = [[0,1],[0,2],[0,3],[1,3],[1,4],[2,3],[2,5],[2,6],[3,4],[3,5],[3,6],[4,6],[5,6]]

Now let's create an oracle that checks each neighboring pair of edges whether they have a different konbini chain or not. 

To do this, we can create a circuit such as the one below to check if district n and its neighboring district n+1 has a different konbini chain (which is the desirable condition) or not. 

<img src="./fig/oracle.png" width="400"></img>

For konbinis that include '0' in the representation of their states (e.g. A(00), B(01), C(10)), you can flip the bit, apply a ccccx gate and then flip it back again. (Note: the white dots below are the control bits that are flipped and are called negative controls.)

By creating this circuit for A to D, you can check whether district n and its adjacent n+1 have a different konbini or not.

<img src="./fig/konbini_oracle.png" width="700"></img>

We can in fact realize the same effect of the above circuit with the one below. (Please see the Tips section at the end of this notebook for details. Furthermore, please check if these set of circuits give you the same results using the truth table.)<br/>
In this solution example, we will use the circuit below to check if adjacent districts have different konbini chains.

<img src="./fig/konbini_oracle_simple.png" width="600"></img>

For our implementation, instead of using ccx or cccx we will use rccx and rcccx which have lower quantum costs. Using rccx and rcccx normally causes a phase shift but this can be compensated by doing an inverse during uncompute. For details about rccx and rcccx, please refer to the paper below. 
[On the advantages of using relative phase Toffolis with an application to multiple control Toffoli optimization](https://arxiv.org/pdf/1508.03273.pdf)

In [5]:
def color_check(qc, data, district1,district2, result):
    data_qubits = [data[district1*2], data[(district1*2)+1], data[district2*2], data[(district2*2)+1]]
    qc.barrier()
    qc.rccx(data_qubits[0], data_qubits[1], result)
    qc.barrier()
    qc.x(data_qubits[2])
    qc.rccx(data_qubits[1], data_qubits[2], result)
    qc.x(data_qubits[2])

    qc.barrier()
    qc.x(data_qubits[2])
    qc.x(data_qubits[3])
    qc.rccx(data_qubits[2], data_qubits[3], result)
    qc.x(data_qubits[2])
    qc.x(data_qubits[3])
    qc.barrier()
    
    qc.x(data_qubits[3])    
    qc.rccx(data_qubits[0], data_qubits[3], result)
    qc.x(data_qubits[3])    
    qc.barrier() 

In [6]:
def color_check_inverse(qc, data, district1,district2, result):
    data_qubits = [data[district1*2], data[(district1*2)+1], data[district2*2], data[(district2*2)+1]]
    
    qc.barrier()
    qc.x(data_qubits[3])    
    qc.rccx(data_qubits[0], data_qubits[3], result)
    qc.x(data_qubits[3])    
    qc.barrier()
    
    qc.barrier()
    qc.x(data_qubits[2])
    qc.x(data_qubits[3])
    qc.rccx(data_qubits[2], data_qubits[3], result)
    qc.x(data_qubits[2])
    qc.x(data_qubits[3])
    qc.barrier()
    
    qc.x(data_qubits[2])
    qc.rccx(data_qubits[1], data_qubits[2], result)
    qc.x(data_qubits[2])
    
    qc.barrier()
    qc.rccx(data_qubits[0], data_qubits[1], result)
    qc.barrier()

In [7]:
def rcccx_inverse(circ,control_0,control_1,control_2,target):
    circ.u2(0, np.pi, target)  
    circ.u1(np.pi / 4, target)  
    circ.cx(control_2, target)
    circ.u1(-np.pi / 4, target)
    circ.u2(0, np.pi, target)
    circ.u1(np.pi / 4, target)
    circ.cx(control_1, target)
    circ.u1(-np.pi / 4, target)
    circ.cx(control_0, target)
    circ.u1(np.pi / 4, target)
    circ.cx(control_1, target)
    circ.u1(-np.pi / 4, target)
    circ.cx(control_0, target)
    circ.u2(0, np.pi, target) 
    circ.u1(np.pi / 4, target)
    circ.cx(control_2, target)
    circ.u1(-np.pi / 4, target)
    circ.u2(0, np.pi, target) 

Now we perform a `color_check()` against all 13 edges. We store the AND result of the `color_check()` retrieved by `mct` into the oracle phase-flip qubit. 

In [8]:
def correct_result(qc, result, anc, register):
    # correct results of color_check() and store in anc
    for i in range(4):
        qc.rcccx(result[3*i], result[3*i+1], result[3*i+2], anc[i])
        qc.barrier()
    
    # use cccccx that has result[13] and anc[0] to anc[3] as its control bits and register as its target bit to flip the phase of correct answers.
    mct_control=[result[12]]+anc
    qc.mct(mct_control, register, result[0:3], mode='basic-dirty-ancilla')
    
    # uncompute corrected results 
    for i in reversed(range(4)):
        rcccx_inverse(qc,result[3*i], result[3*i+1], result[3*i+2], anc[i])
        qc.barrier()

Now we create an `oracle()` by combining `color_check()` and `collect_result()`. `color_check()` gives back 1 when the adjacent districts have the same konbini, so we apply the x gate to initialize `memory_for_result` to $|1>$.

In [9]:
def oracle(qc, data, result, anc, register):
    result_index = 0
    
    #initialize memory_for_result to |1>
    qc.x(result)
        
    #color check for each edge
    for edge in edge_list:
        color_check(qc, data, edge[0],edge[1], result[result_index])
        result_index += 1

    qc.barrier()

    #collect results of each edge
    correct_result(qc, result, anc, register)
    qc.barrier()
    
    #inversed color check
    result_index = 0 
    for edge in edge_list:
        color_check_inverse(qc, data, edge[0],edge[1], result[result_index])
        result_index += 1
    qc.barrier()
    
    #reinitilize memory_for_result to |0>
    qc.x(result)

## Step3: Diffusion

As we learned in week 2, Grover's algorithm is comprised of the following two operators. 
1. Oracle Reflection $U_w$
2. Inversion $U_s$ about the initial uniform superposition $|s>$

<img src="./fig/step2.png" width="700">

In this step, we create a diffusion circuit $U_s$ that reflects about $|s>$

In week 2 we created a uniform superposition by applying the Hadamard gate but in this particular challenge, we have constraints in creating the initial state thus, the diffusion will also need some modification. 

Please remember that the reflection about the initial state $|s>$ can be described as $U_{s} = 2|s> <s| - I $ 

We denote the circuit for creating the superposition of inputs with constraints in step 1 as $U_{const}$ and apply this to our diffusion circuit.

$$U_s = 2|s> <s| - I \\= 2U_{const}|0><0|U_{const}^\dagger - I \\=-U_{const}(I-2|0><0|)U^\dagger$$

We will need a $U_{const}^\dagger$ which is the inverse of $U_{const}$. When we apply $U_{const}^\dagger$ to the circuit, we will get:

In [10]:
def initialize_districts_inverse(qc, qr):
    #district0 A inverse
    qc.x(qr[1])
    qc.ch(qr[0],qr[1])
    qc.ry(-theta,qr[0])

    #district1 B inverse
    qc.ch(qr[2],qr[3])
    qc.ry(-theta,qr[2])

    #district2 A,C inverse
    qc.x(qr[5])
    qc.h(qr[4])

    #district3 A inverse
    qc.x(qr[7])
    qc.ch(qr[6],qr[7])
    qc.ry(-theta,qr[6])

    #district4 B inverse
    qc.ch(qr[8],qr[9])
    qc.ry(-theta,qr[8])

    #district5 D inverse
    qc.x(qr[10])
    qc.ch(qr[10],qr[11])
    qc.ry(-theta,qr[10])

    #district6 D inverse
    qc.x(qr[12])
    qc.ch(qr[12],qr[13])
    qc.ry(-theta,qr[12])

In [11]:
def inversion_about_average(circuit, register):
    """Apply inversion about the average step of Grover's algorithm."""
    initialize_districts_inverse(circuit, register)
    circuit.x(register[0:14])
    
    circuit.h(register[13])
    circuit.mct(register[0:13],register[13],register[14:26])
    circuit.h(register[13])

    circuit.x(register[0:14])
    initialize_districts(circuit, register)

Initialize the phase flip quantum bit and iterate the diffusion 5 times.

In [12]:
# Initialize 
initialize_districts(qc, qr)

# Initialize a qubit for phase flip
qc.x(qr[27])
qc.h(qr[27])

# Grover iteration
for i in range(5):
    oracle(qc, qr[0:14], qr[14:27], qr[28:32], qr[27])
    inversion_about_average(qc,qr)

# Uncompute
qc.h(qr[27])
qc.x(qr[27])

## Step4: Measurement
In this step, we measure the qubits where the districts are mapped to.

In [13]:
%%time
qc.measure(qr[0:14],cr[0:14])

provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')
job = execute(qc, backend=backend, shots=8000, seed_simulator=12345, backend_options={"fusion_enable":True})
result = job.result()
count = result.get_counts()

CPU times: user 18.5 s, sys: 737 ms, total: 19.3 s
Wall time: 1h 19min 20s


The measurement scores grow very large, so we sort the score and extract the top 15 results. 

In [14]:
score_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)
final_score = score_sorted[0:15]
final_score

[('00010111100001', 546),
 ('00011110110001', 531),
 ('01001110110001', 523),
 ('10000001111110', 523),
 ('00010110111101', 515),
 ('01000010111101', 509),
 ('00010110110001', 506),
 ('00101101110010', 501),
 ('10001101110010', 496),
 ('00000001110101', 9),
 ('01010101110001', 8),
 ('00100011110011', 8),
 ('10000001100110', 8),
 ('00010001101111', 8),
 ('01010111110110', 7)]

Here you will see nine results showing amplitude amplification predominantly. We collect the answers with the 9 highest probabilities.

In [15]:
#sort count
count_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)

# collect answers with Top 9 probabilities
ans_list = count_sorted[0:9]

# reverse ans_list
ans_reversed = []
for i in ans_list:
    ans_temp=[i[0][::-1],i[1]]
    ans_reversed.append(ans_temp)

# convert each 2 bits into corresponding color. Add node0(0),node3(1),node8(2) and node11(3)
ans_shaped = []
for j in ans_reversed:
    ans_temp=j[0]
    nodeA = 0
    node0 = int(ans_temp[0] + ans_temp[1], 2)
    node1 = int(ans_temp[2] + ans_temp[3], 2)
    nodeB = 1
    node2 = int(ans_temp[4] + ans_temp[5], 2)
    node3 = int(ans_temp[6] + ans_temp[7], 2)
    node4 = int(ans_temp[8] + ans_temp[9], 2)
    nodeC = 2
    node5 = int(ans_temp[10] + ans_temp[11], 2)
    node6 = int(ans_temp[12] + ans_temp[13], 2)
    nodeD = 3
    nodes_color = str(nodeA) + str(node0) + str(node1) + str(nodeB) + str(node2) + str(node3) + str(node4) + str(nodeC) + str(node5) + str(node6) + str(nodeD) 
    ans_shaped.append([nodes_color,j[1]])
sorted(ans_shaped)

[['01013232013', 496],
 ['01013232103', 501],
 ['01313202013', 523],
 ['02011322203', 546],
 ['02013122203', 506],
 ['02013132023', 523],
 ['02013132203', 531],
 ['02313102023', 509],
 ['02313122203', 515]]

Color the districts based on your results and you can verify that you got it right! Congratulations!
<img src="./fig/all_answers.png" width="600"></img>

In [16]:
import json
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroll the circuit
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(qc) 

# obtain gates
gates=new_circuit.count_ops()
print(gates)

OrderedDict([('u3', 6507), ('cx', 2416), ('barrier', 770), ('measure', 14)])


In [17]:
cost=gates['u3'] + 10*gates['cx']
print(cost)

30667


The quantum cost of the circuit in this particular example is $30667$.

## TIPS
To replace the four ccccx by four ccx in `color_check()`, you can map a 4bit gray code into a hypercube. （Please refer to [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) for details）
<img src="./fig/4bit_hyper_cube.png" width="600"></img>

Let's take a look at this hypercube. Here, the four ccccx gates which were defined to check if a certain konbini chain was present in adjacent districts will each correspond to coloring vertex 0000, 0101, 1010, and 1111 of this hypercube. A single ccx gate will correspond to coloring a single plane of a hypercube so, if you can find out the appropriate combination of planes (note: vertices that share even number of planes cancel out each other and cannot be colored), you can replace your four ccccx gates with four ccx gates. Check and see that the following combination of planes would allow you to color 0000, 0101, 1010, 1111 with the right konbinis (colors). Please note that other vertices are not colored because they share an even number of planes. 
<img src="./fig/hyper_cube_ccx.png" width="600"></img>

Ccx gates for each corresponding plane is as follows:
<img src="./fig/konbini_oracle_hyper_cube.png" width="500"></img>

## Honorary Mentions
The judges would like to make honorary mentions of the following teams which were able to achieve exceptional results.

**The Winning Teams:**
* First Place Team: **Whit3z**: cost = $16613$  
This team has successfully realized a smaller oracle by identifying which districts need only a simple check. From collapsing multiple single-qubits into u3 gates to defining a margolus gate, team Whit3z successfully optimized their circuit to the fine details. Team whit3z was also one of the first teams to submit their code where the cost was initially 164749 but within a week reduced the gate cost to a tenth which was very impressive.  
* Second Place Team: **QunaVillage**: cost = $17053$  
Team QunaVillage reduced the number of qubits for storing district info. and reduced the solution space effectively. Their technique to check multiple edges at once was also very clever.   
* Third Place Team: **IIQ@QIC**: cost = $25490$   
An oracle that simultaneously checks edges that does not share the same node was designed effectively. Optimization is applied across the circuit in detail such as in the initialization, corresponding diffusion, and in minimizing u3 gates. <br/>

======<br/>
**Special Oracle Award** <br/>
Although part of the encoding for initialization was incomplete and was not enough to generate equal probability distributions, we would like to honor and recognize two teams that have shown ingenuity and creativity in how they built their oracles and diffusion functions. The two teams are:<br/>
* Team: **Costs > 100k**: $\approx 15737$  
their oracle design, which focuses on the geometric triangle of the graph, was exceptionally creative. With their oracle, they succeeded reduced the solution space dramatically. That is why the organizers have decided to award this team with a Special Oracle Award. 
* Team: **Sorin**: $\approx 21667$  
Team Sorin uses a unique function `equalsNoCcx()` which provides the same functionality as the color_check() we used in our example. They use only a cx gate and a x gate in this function which returns the district information back into the storage qubit based on the comparison result. This approach allows team Sorin to build an effective oracle. 

Judges would like to honor the above teams as well as the teams that made it into the top ten teams. Congratulations! <br/>
**Check out the top team's submissions and write-ups [here](https://github.com/quantum-challenge/2019/tree/master/top%20ten%20submissions).**  (*We have a few latest additions in terms of the top ten. We will be reaching out to those teams and asking them to upload their submission and write-ups soon.*)

Over 700 people joined IBM Quantum Challenge. We were amazed by the ingenuity and creativity of each team especially the teams mentioned above who achieved lower quantum costs beyond the judges' expectation. We were also happy to know that there were many individuals out there who joined this challenge perhaps not necessarily to compete but to learn quantum computation and new techniques. We hope you had as much fun as we did on this wild ride. 

Once again, a **BIG THANK YOU to all the teams and individuals who have participated in this challenge.** <br/>Until we meet again. <br/>

Judges and organizers, IBM Quantum Challenge